# Model Architecture


<img src="u-net-architecture.png">

Image taken from the original paper here: https://arxiv.org/abs/1505.04597

In [3]:
import torch
import torchvision.transforms.functional as F
import torch.nn as nn

In [4]:
FEATURES = [64, 128, 256, 512]

In [5]:
class PairConv(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(PairConv, self).__init__()
        self.conv = nn.Sequential(
                nn.Conv2d(in_channels, 
                          out_channels, 
                          kernel_size = 3,
                          stride = 1, 
                          padding = 1,
                          bias = False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace = True),
                nn.Conv2d(out_channels, 
                          out_channels, 
                          kernel_size = 3,
                          stride = 1, 
                          padding = 1,
                          bias = False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace = True)
            )
        
    def forward(self, x):
        return self.conv(x)

In [6]:
class UNET(nn.Module):

    def __init__(self, in_channels = 3, out_channels = 1, features = FEATURES):
        super(UNET, self).__init__()
        self.contracting_path = nn.ModuleList()
        self.expansive_path = nn.ModuleList()
        self.max_pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        for feature in features:
            self.contracting_path.append(PairConv(in_channels, feature))
            in_channels = feature

        for feature in reversed(features):
            self.expansive_path.append(
            nn.ConvTranspose2d(
                            feature*2,
                            feature,
                            kernel_size= 2,
                            stride = 2
                        )
                )
            self.expansive_path.append(PairConv(feature * 2, feature))
        
        self.bottom = PairConv(features[-1], features[-1] * 2)
        self.out_conv = nn.Conv2d(features[0], out_channels,kernel_size = 1)

    def forward(self, x):

        skip_connections = []

        for down in self.contracting_path:
            x = down(x)
            skip_connections.append(x)
            x = self.max_pool(x)

        x = self.bottom(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.expansive_path), 2):
            x = self.expansive_path[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = F.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.expansive_path[idx+1](concat_skip)

        return self.out_conv(x)



In [7]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset


In [8]:
class ImagesDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform = None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(img_dir)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.jpg', '_mask.gif'))
        img = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32) #Greyscale
        mask[mask == 255] = 1.0

        if self.transform is not None:
            augmentations = self.transform(img=img, mask=mask)
            image = augmentations["img"]
            mask = augmentations["mask"]
        
        return image, mask

## Training

In [9]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2

### Hyperparameters

In [11]:
LEARNING_RATE = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_IMG_PATH = "data/train_img/"
TRAIN_MASK_PATH = "data/train_mask/"
VAL_IMG_PATH = "data/val_img/"
VAL_MASK_PATH = "data/val_mask/"
BATCH_SIZE = 32
NUM_EPOCHS = 50
NUM_WORKERS = 2
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 240
PIN_MEMORY = True
LOAD_MODEL = False

In [12]:
def train(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_id, (data, target) in enumerate(loop):
        data = data.to(DEVICE)
        targets = targets.float().unsqueeze(1).to(DEVICE)

        #forward
        with torch.cuda.amp.autocast():
            prediction = model(data)
            loss = loss_fn(prediction, targets)
        #backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_postfix(loss=loss.item())


In [13]:
def augment():

    train_augmentation = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=95, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),

                ],
            )
    
    val_augmentation = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),

                ],
            )

In [ ]:
model = UNET(in_channels=4, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss() #No Sigmoid in ouput of model
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(NUM_EPOCHS):
    continue